[`pidgin.weave`](weave.md.ipynb) allows code to be execute with the display function.

In [1]:
    import ast, jinja2, sys, nbconvert, nbformat, contextlib, io, emoji, collections, IPython, mistune, doctest, re, traitlets, textwrap, types, typing as t, importlib, importnb,  jinja2, builtins, jsonschema
    with __import__('importnb').Notebook():
        try:
            from . import tangle
        except:
            from pidgin import tangle
            
    with tangle.Pidgin(): 
        try:
            from . import shell, formatter, specification
            from .specification import template, markdown, transform_cell, emojis, json, testing
        except:
            from pidgin import shell, formatter, specification
            from pidgin.specification import template, markdown, transform_cell, emojis, json, testing

    if __name__ == '__main__':
        %reload_ext pidgin

In [2]:
    def load_ipython_extension(shell: IPython.InteractiveShell):
>>> assert '_document_' in sys.modules
>>> assert '_test_' in sys.modules
        
        from . import formatter
        def make_module(str):        
            try:  return importlib.import_module(str)
            except:  return sys.modules.__setitem__(str, types.ModuleType(str)) or sys.modules[str]
        document, test = map(make_module, '_document_ _test_'.split())
        test = shell.test = testing.Doctest(user_module=test, parent=shell, parents=[shell])
        shell.markdown = markdown.Markdown(user_module=document, parent=shell, parents=[test, shell])
        shell.template = template.Template(parent=shell, parents=[shell.markdown, test, shell])
        shell.weave = Weave(user_module=document, parent=shell, parents=[test, shell], markdown=shell.markdown, template=shell.template, test=test, )
        
        shell.display_formatter.formatters['text/html'].for_type(str, format_http)
        shell.events.register('post_run_cell', test)
        shell.events.register('post_run_cell', shell.weave)
        shell.enable_html_pager = True

1,def load_ipython_extension(shell: IPython.InteractiveShell):
5 6 7 8 9 10 11 12 13 14 15 16 17 18,"from . import formatter def make_module(str): try: return importlib.import_module(str) except: return sys.modules.__setitem__(str, types.ModuleType(str)) or sys.modules[str] document, test = map(make_module, '_document_ _test_'.split()) test = shell.test = testing.Doctest(user_module=test, parent=shell, parents=[shell]) shell.markdown = markdown.Markdown(user_module=document, parent=shell, parents=[test, shell]) shell.template = template.Template(parent=shell, parents=[shell.markdown, test, shell]) shell.weave = Weave(user_module=document, parent=shell, parents=[test, shell], markdown=shell.markdown, template=shell.template, test=test, ) shell.display_formatter.formatters['text/html'].for_type(str, format_http) shell.events.register('post_run_cell', test) shell.events.register('post_run_cell', shell.weave) shell.enable_html_pager = True"


In [3]:
    def unload_ipython_extension(shell: IPython.InteractiveShell):
        shell.events.unregister('post_run_cell', shell.test)
        shell.events.unregister('post_run_cell', shell.weave)

In [4]:
    def format_http(str): 
>>> shell = IPython.get_ipython()
>>> shell.display_formatter.format("http://abc")
({'text/plain': "'http://abc'", 'text/html': '<p><a href="http://abc">http://abc</a></p>\n'}, {})
>>> shell.weave.display_formatter.format("http://abc")
(...IPython.lib.display.IFrame...)
        
        if str.startswith(('http:', 'https:')): return mistune.markdown(str)

1,def format_http(str):
8,"if str.startswith(('http:', 'https:')): return mistune.markdown(str)"


In [5]:
    class Weave(formatter.Formatter):
        test = traitlets.Instance(IPython.InteractiveShell)
        markdown = traitlets.Instance(IPython.InteractiveShell, allow_none=True)
        template = traitlets.Instance(IPython.InteractiveShell)
        
        __call__ = shell.Shell.__call__
        
        def __init__(Weave, *args, **kwargs):
            formatter.Formatter.__init__(Weave, *args, **kwargs)
            Weave.environment = Weave.template.environment
            
        def run_cell(Weave, text: str, *args, **kwargs):
            if isinstance(text, IPython.core.interactiveshell.ExecutionResult): 
                text = text.info.raw_cell            
            for parent in (Weave.parents or []): vars(Weave.user_module).update(parent.user_ns)
            result = super().run_cell(text, *args, **kwargs)
            return result
        
        def front_matter(Weave, text):
            try:
                import yaml
            except: 
                import ruamel.yaml as yaml
                
            if text.startswith(('---',)):
                text = text.lstrip('---')
                text, sep, end = text.partition('\n---\n')
                if sep:
                    return end, dict(yaml.safe_load(__import__('io').StringIO(Weave.template.render(text))))
            return text, {}
        
        def render(Weave, text, **metadata):
            
            text, metadata = Weave.front_matter(text)
            if text in Weave.user_ns:
                if not isinstance(Weave.user_ns[text], str):
                    return Weave.parent.display_formatter.format(Weave.user_ns[text])
                text  = Weave.user_ns[text]
            text = json.expand(text, Weave.user_ns.get('__context__', {}))
            if len(text.splitlines()) == 1 and text.startswith(('http:', 'https:')):
                return IPython.display.IFrame(text, "100%", "600px", metadata=metadata)
            text = specification.emojis.emojize(text)
            text = Weave.template.render(text, **metadata)
            text = Weave.markdown.render(text)   
            return IPython.display.HTML(text, metadata=metadata)
        
        def publish(Template, text):
            return super().publish(text) and not only_contains_block_code(text, specification.transform_cell.markdown_to_python(text))

In [6]:
    def only_contains_block_code(raw: str, source: str) -> bool: 
>>> assert only_contains_block_code("""f""", """f""")
>>> assert only_contains_block_code("""%%magic""", """%%magic""")
>>> assert not only_contains_block_code("""Markdown""", """'Markdown'""")

        return raw.startswith('%%') or list(map(str.rstrip, textwrap.dedent(raw).splitlines())) == list(map(str.rstrip, textwrap.dedent(source).splitlines()))

1,"def only_contains_block_code(raw: str, source: str) -> bool:"
6,"return raw.startswith('%%') or list(map(str.rstrip, textwrap.dedent(raw).splitlines())) == list(map(str.rstrip, textwrap.dedent(source).splitlines()))"


In [7]:
    def do_publish(str) -> bool:
>>> assert do_publish("""___beach_with_umbrella___""")
>>> assert not do_publish("""//___bear_face___""")
>>> assert not do_publish("""
... ___no_bicycles___""")
        
        display = bool(str.partition('\n')[0].strip()) and not str.startswith('//')            
        source = transform_cell.markdown_to_python(str)
        display = display & (not bool(only_contains_block_code(str, source)))
        try:
            if str.strip() != ast.literal_eval(source.rstrip().rstrip(';')).strip(): display = False
        finally: return display

1,def do_publish(str) -> bool:
7 8 9 10 11 12,"display = bool(str.partition('\n')[0].strip()) and not str.startswith('//') source = transform_cell.markdown_to_python(str) display = display & (not bool(only_contains_block_code(str, source))) try: if str.strip() != ast.literal_eval(source.rstrip().rstrip(';')).strip(): display = False finally: return display"
